## BCL2FASQ COMMAND

In [ ]:
#separates differenlty indexed samples. multiplexes i7. NOT dempultiplexing the wells but the lanes on 10x
#there will be two sublibraries of the i7 indeces. Nextset
#the scifi5 o_DT are the main seq containing cdna, WP are the AAVs
#bcl2fastq -i ./Data/Intensities/BaseCalls/ -R ./ -o ./demultiplexed/ --use-bases-mask Y26,I8,Y17,Y115 --sample-sheet ../20220114_SampleSheet_HJCNYAFX3.csv -r 8 -p 24 -w 8 --no-lane-splitting --minimum-trimmed-read-length 10 --mask-short-adapter-reads 10 --ignore-missing-bcl --create-fastq-for-index-reads --ignore-missing-filter --ignore-missing-position

In [1]:
import os
import warnings
from subprocess import call
import subprocess
import glob
import pandas as pd
import tempfile
import ntpath
import numpy as np
import matplotlib.pyplot as plt
import re


In [2]:
#in_path = "'/media/Dropbox/MNM network files/OrganizedSeqFiles/snRNA-aSyn/MNM220114/220114_NB502004_0177_AHJCNYAFX3/demultiplexed/SciFi_10X_Pilot5_Validation_R2'"
#out_path = "'/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/output_10X_pilot5_r2'"
#base_path = '/tmp/indata'
#export_path = '/tmp/outdata'
in_path = "'/media/data/AtteR/210517_A00681_0370_AHF2JHDRXY/demultiplexed'"
out_path = "'/media/data/AtteR/scifi-analysis/pilot1_output'"

in_path = "/media/data/AtteR/scifi-analysis/scifi6/220222_NB502004_0181_AHK7L2AFX3/demultiplexed/SciFi_10X_Pilot6_Pooled"
out_path = "/media/data/AtteR/scifi-analysis/scifi6/output"
base_path = '/tmp/indata'
export_path = '/tmp/outdata'
output_data2 = '/media/data/AtteR/scifi-analysis/scifi6' 
output_tables = '/media/data/AtteR/scifi-analysis/summary_tables'


In [3]:
#the symlinks are generated to ensure path without spaces which bbmap cannot handle
!mkdir $out_path
#!rm $base_path
!ln -s $in_path $base_path
!rm $export_path
!ln -s $out_path $export_path
base_path = base_path +'/'
export_path = export_path + '/'


mkdir: cannot create directory ‘/media/data/AtteR/scifi-analysis/scifi6/output’: File exists
ln: failed to create symbolic link '/tmp/indata/SciFi_10X_Pilot6_Pooled': File exists


### Fuse Scifi and Bead barcode reads

In [6]:
def fuseBCs(sample_id) :
    print(sample_id) 
    revCompR1 = "/media/data/AtteR/scifi-analysis/scifi6/220222_NB502004_0181_AHK7L2AFX3/demultiplexed/SciFi_10X_Pilot6_Pooled/"+ os.path.basename(sample_id).replace('_R1_001.fastq.gz','_RC_R1_001.fastq.gz')

    #revCompR1 = '/media/data/AtteR/scifi-analysis/'+ os.path.basename(sample_id).replace('_R1_001.fastq.gz','_RC_R1_001.fastq.gz')
    rc_call= "reformat.sh overwrite=t rcomp=t in='"+ sample_id +"' out='"+revCompR1+"'"
    print(rc_call)
    !$rc_call
    #reverse complement of the read1 needed prior to fusing it with with read2
    inPathR2 = base_path + ntpath.basename(sample_id).replace('_R1_001.fastq.gz','_R2_001.fastq.gz')
    outPath = base_path + ntpath.basename(sample_id).replace('_R1_001.fastq.gz','_R21_001.fastq.gz')
    fuse_call= "/usr/local/bin/bbmap/fuse.sh in1='"+ inPathR2 +"' in2='"+revCompR1+"' out='"+outPath+"' overwrite=t interleaved=f fusepairs=t pad=0"
    !$fuse_call  #BBMap is a splice-aware global aligner
    os.remove(revCompR1)
    print(fuse_call)
    print("outPath: " + outPath)
    


In [4]:
samples_list = sorted(glob.glob(base_path+'*R1*.fastq.gz'))
samples_list= [item for item in samples_list if 'Undetermined' not in item] 
#samples_list= [item for item in samples_list if 'EnrichedBCs' not in item] 

#samples_list= [item for item in samples_list if 'BC' in item]
print(samples_list)
for sample_id in samples_list:
    print(ntpath.basename(sample_id))

['/tmp/indata/SciFi_Pool1-2_EnrichedBCs_All_R1_001.fastq.gz', '/tmp/indata/SciFi_Pool1-2_EnrichedBCs_R1_001.fastq.gz', '/tmp/indata/SciFi_Pool1-2_EnrichedBCs_RC_R1_001.fastq.gz', '/tmp/indata/SciFi_new_cDNA_Pool1_R1_001.fastq.gz', '/tmp/indata/SciFi_old_cDNA_Pool2_R1_001.fastq.gz']
SciFi_Pool1-2_EnrichedBCs_All_R1_001.fastq.gz
SciFi_Pool1-2_EnrichedBCs_R1_001.fastq.gz
SciFi_Pool1-2_EnrichedBCs_RC_R1_001.fastq.gz
SciFi_new_cDNA_Pool1_R1_001.fastq.gz
SciFi_old_cDNA_Pool2_R1_001.fastq.gz


In [7]:
[fuseBCs(sample_id) for sample_id in samples_list]

/tmp/indata/SciFi_Pool1-2_EnrichedBCs_All_R1_001.fastq.gz
reformat.sh overwrite=t rcomp=t in='/tmp/indata/SciFi_Pool1-2_EnrichedBCs_All_R1_001.fastq.gz' out='/media/data/AtteR/scifi-analysis/scifi6/220222_NB502004_0181_AHK7L2AFX3/demultiplexed/SciFi_10X_Pilot6_Pooled/SciFi_Pool1-2_EnrichedBCs_All_RC_R1_001.fastq.gz'
java -ea -Xms300m -cp /usr/local/bin/bbmap/current/ jgi.ReformatReads overwrite=t rcomp=t in=/tmp/indata/SciFi_Pool1-2_EnrichedBCs_All_R1_001.fastq.gz out=/media/data/AtteR/scifi-analysis/scifi6/220222_NB502004_0181_AHK7L2AFX3/demultiplexed/SciFi_10X_Pilot6_Pooled/SciFi_Pool1-2_EnrichedBCs_All_RC_R1_001.fastq.gz
Executing jgi.ReformatReads [overwrite=t, rcomp=t, in=/tmp/indata/SciFi_Pool1-2_EnrichedBCs_All_R1_001.fastq.gz, out=/media/data/AtteR/scifi-analysis/scifi6/220222_NB502004_0181_AHK7L2AFX3/demultiplexed/SciFi_10X_Pilot6_Pooled/SciFi_Pool1-2_EnrichedBCs_All_RC_R1_001.fastq.gz]

Set RCOMP to true
Input is being processed as unpaired
Input:                  	84309009 r

FileNotFoundError: [Errno 2] No such file or directory: '/media/data/AtteR/scifi-analysis/scifi6/220222_NB502004_0181_AHK7L2AFX3/demultiplexed/SciFi_10X_Pilot6_Pooled/SciFi_Pool1-2_EnrichedBCs_All_RC_RC_R1_001.fastq.gz'

In [ ]:
'''
SciFi_Kapa_AAV_aSyn_S6_RC_R1_001.fastq.gz, 
SciFi_Kapa_AAV_aSyn_S6_R21_001.fastq.gz
'''

In [5]:
samples_list = sorted(glob.glob(base_path+'*R1*.fastq.gz'))
#samples_list= [item for item in samples_list if 'Undetermined' not in item]
#samples_list= [item for item in samples_list if 'BC' in item]
samples_list

['/tmp/indata/SciFi_Pool1-2_EnrichedBCs_All_R1_001.fastq.gz',
 '/tmp/indata/SciFi_Pool1-2_EnrichedBCs_R1_001.fastq.gz',
 '/tmp/indata/SciFi_Pool1-2_EnrichedBCs_RC_R1_001.fastq.gz',
 '/tmp/indata/SciFi_new_cDNA_Pool1_R1_001.fastq.gz',
 '/tmp/indata/SciFi_old_cDNA_Pool2_R1_001.fastq.gz']

In [6]:
samples_list = sorted(glob.glob(base_path+'*R21*.fastq.gz'))
#samples_list= [item for item in samples_list if 'Undetermined' not in item]
samples_list= [item for item in samples_list if 'EnrichedBCs' not in item]
samples_list

['/tmp/indata/SciFi_new_cDNA_Pool1_R21_001.fastq.gz',
 '/tmp/indata/SciFi_old_cDNA_Pool2_R21_001.fastq.gz']

### Trim polyA

In [10]:
def runTrimPA(sample_id) :
    print(sample_id)
    outPath = export_path + ntpath.basename(sample_id).replace('R21_001.fastq.gz','R21_trimPA.fastq.gz')
    samplePair = base_path + ntpath.basename(sample_id).replace('R21_001.fastq.gz','R3_001.fastq.gz')
    outPathR3 = export_path + ntpath.basename(samplePair).replace('R3_001.fastq.gz','R3_trimPA.fastq.gz')
    cutAdapt_call="cutadapt --cores=0 --nextseq-trim=25 --minimum-length 25 -g GCTGCACCGTCAGA -O 12 -a 'A{30}N{104}' -o "+ outPathR3 + " -p " + outPath + " " + samplePair + " " + sample_id
    print(cutAdapt_call)
    !$cutAdapt_call 

In [7]:
#samples_list = sorted(glob.glob(base_path+'*_R21_001.fastq.gz'))
#For scifi6
samples_list = sorted(glob.glob(base_path+'*_R21_001.fastq.gz'))
#samples_list= [item for item in samples_list if 'EnrichedBCs' not in item]
samples_list
#!pip3 install cutadapt

['/tmp/indata/SciFi_Pool1-2_EnrichedBCs_All_R21_001.fastq.gz',
 '/tmp/indata/SciFi_Pool1-2_EnrichedBCs_All_RC_R21_001.fastq.gz',
 '/tmp/indata/SciFi_Pool1-2_EnrichedBCs_R21_001.fastq.gz',
 '/tmp/indata/SciFi_new_cDNA_Pool1_R21_001.fastq.gz',
 '/tmp/indata/SciFi_old_cDNA_Pool2_R21_001.fastq.gz']

In [12]:

#earlier was samples_list[2:] and initially we had enriched bcs (3 files so one of them got included into the loop)in
#[runTrimPA(sample_id) for sample_id in samples_list[2:]]
[runTrimPA(sample_id) for sample_id in samples_list]

/tmp/indata/SciFi_Pool1-2_EnrichedBCs_All_R21_001.fastq.gz
cutadapt --cores=0 --nextseq-trim=25 --minimum-length 25 -g GCTGCACCGTCAGA -O 12 -a 'A{30}N{104}' -o /tmp/outdata/SciFi_Pool1-2_EnrichedBCs_All_R3_trimPA.fastq.gz -p /tmp/outdata/SciFi_Pool1-2_EnrichedBCs_All_R21_trimPA.fastq.gz /tmp/indata/SciFi_Pool1-2_EnrichedBCs_All_R3_001.fastq.gz /tmp/indata/SciFi_Pool1-2_EnrichedBCs_All_R21_001.fastq.gz
This is cutadapt 3.5 with Python 3.8.5
Command line parameters: --cores=0 --nextseq-trim=25 --minimum-length 25 -g GCTGCACCGTCAGA -O 12 -a A{30}N{104} -o /tmp/outdata/SciFi_Pool1-2_EnrichedBCs_All_R3_trimPA.fastq.gz -p /tmp/outdata/SciFi_Pool1-2_EnrichedBCs_All_R21_trimPA.fastq.gz /tmp/indata/SciFi_Pool1-2_EnrichedBCs_All_R3_001.fastq.gz /tmp/indata/SciFi_Pool1-2_EnrichedBCs_All_R21_001.fastq.gz
Processing reads on 48 cores in paired-end mode ...
Done           00:07:20    84,309,009 reads @   5.2 µs/read;  11.49 M reads/minute
Finished in 440.24 s (5 µs/read; 11.49 M reads/minute).

=== 

Processing reads on 48 cores in paired-end mode ...
ERROR: Traceback (most recent call last):
  File "/home/lcadmin/.local/lib/python3.8/site-packages/cutadapt/pipeline.py", line 567, in run
    with self._opener.xopen(self.path, 'rb') as f:
  File "/home/lcadmin/.local/lib/python3.8/site-packages/cutadapt/utils.py", line 192, in xopen
    f = open_raise_limit(
  File "/home/lcadmin/.local/lib/python3.8/site-packages/cutadapt/utils.py", line 51, in open_raise_limit
    f = func(*args, **kwargs)
  File "/home/lcadmin/miniconda3/envs/scifi-analysis/lib/python3.8/site-packages/xopen/__init__.py", line 821, in xopen
    opened_file = _open_gz(filename, mode, compresslevel, threads, **text_mode_kwargs)
  File "/home/lcadmin/miniconda3/envs/scifi-analysis/lib/python3.8/site-packages/xopen/__init__.py", line 695, in _open_gz
    return igzip.open(filename, mode, **text_mode_kwargs)
  File "/home/lcadmin/miniconda3/envs/scifi-analysis/lib/python3.8/site-packages/isal/igzip.py", line 92, in ope

ERROR: Traceback (most recent call last):
  File "/home/lcadmin/.local/lib/python3.8/site-packages/cutadapt/pipeline.py", line 567, in run
    with self._opener.xopen(self.path, 'rb') as f:
  File "/home/lcadmin/.local/lib/python3.8/site-packages/cutadapt/utils.py", line 192, in xopen
    f = open_raise_limit(
  File "/home/lcadmin/.local/lib/python3.8/site-packages/cutadapt/utils.py", line 51, in open_raise_limit
    f = func(*args, **kwargs)
  File "/home/lcadmin/miniconda3/envs/scifi-analysis/lib/python3.8/site-packages/xopen/__init__.py", line 821, in xopen
    opened_file = _open_gz(filename, mode, compresslevel, threads, **text_mode_kwargs)
  File "/home/lcadmin/miniconda3/envs/scifi-analysis/lib/python3.8/site-packages/xopen/__init__.py", line 695, in _open_gz
    return igzip.open(filename, mode, **text_mode_kwargs)
  File "/home/lcadmin/miniconda3/envs/scifi-analysis/lib/python3.8/site-packages/isal/igzip.py", line 92, in open
    binary_file = IGzipFile(filename, gz_mode, com

/tmp/indata/SciFi_Pool1-2_EnrichedBCs_R21_001.fastq.gz
cutadapt --cores=0 --nextseq-trim=25 --minimum-length 25 -g GCTGCACCGTCAGA -O 12 -a 'A{30}N{104}' -o /tmp/outdata/SciFi_Pool1-2_EnrichedBCs_R3_trimPA.fastq.gz -p /tmp/outdata/SciFi_Pool1-2_EnrichedBCs_R21_trimPA.fastq.gz /tmp/indata/SciFi_Pool1-2_EnrichedBCs_R3_001.fastq.gz /tmp/indata/SciFi_Pool1-2_EnrichedBCs_R21_001.fastq.gz
This is cutadapt 3.5 with Python 3.8.5
Command line parameters: --cores=0 --nextseq-trim=25 --minimum-length 25 -g GCTGCACCGTCAGA -O 12 -a A{30}N{104} -o /tmp/outdata/SciFi_Pool1-2_EnrichedBCs_R3_trimPA.fastq.gz -p /tmp/outdata/SciFi_Pool1-2_EnrichedBCs_R21_trimPA.fastq.gz /tmp/indata/SciFi_Pool1-2_EnrichedBCs_R3_001.fastq.gz /tmp/indata/SciFi_Pool1-2_EnrichedBCs_R21_001.fastq.gz
Processing reads on 48 cores in paired-end mode ...
Done           00:01:20    15,845,078 reads @   5.1 µs/read;  11.76 M reads/minute
Finished in 80.89 s (5 µs/read; 11.75 M reads/minute).

=== Summary ===

Total read pairs process

/tmp/indata/SciFi_old_cDNA_Pool2_R21_001.fastq.gz
cutadapt --cores=0 --nextseq-trim=25 --minimum-length 25 -g GCTGCACCGTCAGA -O 12 -a 'A{30}N{104}' -o /tmp/outdata/SciFi_old_cDNA_Pool2_R3_trimPA.fastq.gz -p /tmp/outdata/SciFi_old_cDNA_Pool2_R21_trimPA.fastq.gz /tmp/indata/SciFi_old_cDNA_Pool2_R3_001.fastq.gz /tmp/indata/SciFi_old_cDNA_Pool2_R21_001.fastq.gz
This is cutadapt 3.5 with Python 3.8.5
Command line parameters: --cores=0 --nextseq-trim=25 --minimum-length 25 -g GCTGCACCGTCAGA -O 12 -a A{30}N{104} -o /tmp/outdata/SciFi_old_cDNA_Pool2_R3_trimPA.fastq.gz -p /tmp/outdata/SciFi_old_cDNA_Pool2_R21_trimPA.fastq.gz /tmp/indata/SciFi_old_cDNA_Pool2_R3_001.fastq.gz /tmp/indata/SciFi_old_cDNA_Pool2_R21_001.fastq.gz
Processing reads on 48 cores in paired-end mode ...
Done           00:01:39    19,786,409 reads @   5.0 µs/read;  11.96 M reads/minute
Finished in 99.28 s (5 µs/read; 11.96 M reads/minute).

=== Summary ===

Total read pairs processed:         19,786,409
  Read 1 with adapter:

[None, None, None, None, None]

In [25]:
import shutil

In [8]:
samples_list = sorted(glob.glob(export_path+'*R3_trimPA.fastq.gz'))

print(samples_list)

samples_list = sorted(glob.glob(export_path+'*_trimPA.fastq.gz'))
'''
#copy the file from temp dir for later use as a security check
save_path = output_data2 + "/trimmed"
for trimd_file in samples_list:
    target_path = save_path + "/" + ntpath.basename(trimd_file)
    print(target_path)
    shutil.copyfile(trimd_file, target_path)
'''

['/tmp/outdata/SciFi_Pool1-2_EnrichedBCs_All_R3_trimPA.fastq.gz', '/tmp/outdata/SciFi_Pool1-2_EnrichedBCs_All_RC_R3_trimPA.fastq.gz', '/tmp/outdata/SciFi_Pool1-2_EnrichedBCs_R3_trimPA.fastq.gz', '/tmp/outdata/SciFi_new_cDNA_Pool1_R3_trimPA.fastq.gz', '/tmp/outdata/SciFi_old_cDNA_Pool2_R3_trimPA.fastq.gz']


'\n#copy the file from temp dir for later use as a security check\nsave_path = output_data2 + "/trimmed"\nfor trimd_file in samples_list:\n    target_path = save_path + "/" + ntpath.basename(trimd_file)\n    print(target_path)\n    shutil.copyfile(trimd_file, target_path)\n'

## Demultiplex Scifi

In [9]:
#R21 used for demultiplexing - to get the well 

#scifi5 gfp pos i.e. DA cell wells, scifi 6, 7, 8 gfp neg

'''
nuclei taken from one animal and fac sorted. gfp pos nuclei put into one well, the other ones were put into different well and this 
was then split into 3 different wells to test different primer reactions. the primers found in index.fasta file
RT index of the amplicon should correspond to the index array

make sure your file does not contain the rev complements of the cell barcodes. the revcomp was used so that it can be used with the reagents
in novaseq. 

solo

'''

samples_list = sorted(glob.glob(export_path+'*R21_trimPA.fastq.gz'))
#samples_list= [item for item in samples_list if 'Undetermined' not in item]
#samples_list= [item for item in samples_list if 'BC' in item]
samples_list

['/tmp/outdata/SciFi_Pool1-2_EnrichedBCs_All_R21_trimPA.fastq.gz',
 '/tmp/outdata/SciFi_Pool1-2_EnrichedBCs_All_RC_R21_trimPA.fastq.gz',
 '/tmp/outdata/SciFi_Pool1-2_EnrichedBCs_R21_trimPA.fastq.gz',
 '/tmp/outdata/SciFi_new_cDNA_Pool1_R21_trimPA.fastq.gz',
 '/tmp/outdata/SciFi_old_cDNA_Pool2_R21_trimPA.fastq.gz']

In [10]:

def deMultiFuse(sample_id) :
    print(sample_id)
    #sample_id = samples_list[6]
    inPathR3 = export_path + ntpath.basename(sample_id).replace('_R21_trimPA.fastq.gz','_R3_trimPA.fastq.gz')
    outPathR1 = export_path + ntpath.basename(sample_id).replace('_R21_trimPA.fastq.gz','_trimmed-{name}_R21.fastq.gz')
    outPathR3 = export_path + ntpath.basename(sample_id).replace('_R21_trimPA.fastq.gz','_trimmed-{name}_R3.fastq.gz')
    trimLog = export_path + ntpath.basename(sample_id).replace('_R21_trimPA.fastq.gz','_trimLog.tsv')
    param = "-e 0.15 --cores=0 --action none -a file:\'" + output_data2 + "/Scifi6_index.fasta\' --report=minimal"
    #param explanations: e, 0= max error rate and min. overlap for the adapters
    call_sequence = "cutadapt "+param+" -o \'"+outPathR1+"\' -p \'" + outPathR3 + "\' \'"+ sample_id + "\' \'" + inPathR3 +"\' > \'" + trimLog +"\'"
  #  print(call_sequence)
    !($call_sequence)      

In [ ]:
[deMultiFuse(sample_id) for sample_id in samples_list]

/tmp/outdata/SciFi_Pool1-2_EnrichedBCs_All_R21_trimPA.fastq.gz
[         8<-] 00:01:06    19,111,902 reads @   3.3 µs/read;  18.17 M reads/minute

### Generate a QC table

In [29]:
qc_list = sorted(glob.glob(export_path+'*_trimLog.tsv'))
allSampleTable = pd.DataFrame()

for qc_file in qc_list :
    df=pd.read_csv(qc_file, sep='\t') # , header=0, index_col=0,skiprows=None
    df = df.set_index(pd.Series(ntpath.basename(qc_file).replace('_trimLog.tsv','')))
    allSampleTable = pd.concat([allSampleTable, df], axis=0)

allSampleTable

,status,in_reads,in_bp,too_short,too_long,too_many_n,out_reads,w/adapters,qualtrim_bp,out_bp,w/adapters2,qualtrim2_bp,out2_bp
SciFi_new_cDNA_Pool1,WARN,47181203,6950928856,0,0,0,47181203,18326779,0,2000148204,0,0,4950780652
SciFi_old_cDNA_Pool2,OK,19005056,2766519333,0,0,0,19005056,17819921,0,816437301,0,0,1950082032


## Remove false cDNA indexes

In [30]:
#removal of sequencing artifacts
samples_list = sorted(glob.glob(export_path+'*trimmed*.fastq.gz'))
for sample_file in samples_list :
    print(sample_file)
    result = subprocess.run(['pigz -dc '+ sample_file+ ' | wc -l'], stdout=subprocess.PIPE, shell=True)
    sampleReads = int(int(result.stdout)/4)
    if sampleReads < 4000 :
        os.remove(sample_file)
    print(sampleReads)

/tmp/outdata/SciFi_new_cDNA_Pool1_trimmed-SciFi5_WP_R21.fastq.gz
5379164
/tmp/outdata/SciFi_new_cDNA_Pool1_trimmed-SciFi5_WP_R3.fastq.gz
5379164
/tmp/outdata/SciFi_new_cDNA_Pool1_trimmed-SciFi5_oDT_R21.fastq.gz
46326
/tmp/outdata/SciFi_new_cDNA_Pool1_trimmed-SciFi5_oDT_R3.fastq.gz
46326
/tmp/outdata/SciFi_new_cDNA_Pool1_trimmed-SciFi6_WP_R21.fastq.gz
30490
/tmp/outdata/SciFi_new_cDNA_Pool1_trimmed-SciFi6_WP_R3.fastq.gz
30490
/tmp/outdata/SciFi_new_cDNA_Pool1_trimmed-SciFi6_oDT_R21.fastq.gz
163191
/tmp/outdata/SciFi_new_cDNA_Pool1_trimmed-SciFi6_oDT_R3.fastq.gz
163191
/tmp/outdata/SciFi_new_cDNA_Pool1_trimmed-SciFi7_WP_R21.fastq.gz
51268
/tmp/outdata/SciFi_new_cDNA_Pool1_trimmed-SciFi7_WP_R3.fastq.gz
51268
/tmp/outdata/SciFi_new_cDNA_Pool1_trimmed-SciFi7_oDT_R21.fastq.gz
27172
/tmp/outdata/SciFi_new_cDNA_Pool1_trimmed-SciFi7_oDT_R3.fastq.gz
27172
/tmp/outdata/SciFi_new_cDNA_Pool1_trimmed-SciFi8_WP_R21.fastq.gz
77589
/tmp/outdata/SciFi_new_cDNA_Pool1_trimmed-SciFi8_WP_R3.fastq.gz
77589
/

### List selected samples

In [30]:
samples_list = sorted(glob.glob(export_path+'*trimmed*R21.fastq.gz'))
for sample_file in samples_list :
    print(sample_file)
    #pigz -d decompressed the sample file and then we provide the number of lines 
    result = subprocess.run(['pigz -dc '+ sample_file+ ' | wc -l'], stdout=subprocess.PIPE, shell=True)
    sampleReads = int(int(result.stdout)/4)
    print(sampleReads)

/tmp/outdata/Scifi_library_2_S2_trimmed-SciFi5_WP_R21.fastq.gz
11607682
/tmp/outdata/Scifi_library_2_S2_trimmed-SciFi5_oDT_R21.fastq.gz
195116926
/tmp/outdata/Scifi_library_2_S2_trimmed-SciFi6_WP_R21.fastq.gz
3590030
/tmp/outdata/Scifi_library_2_S2_trimmed-SciFi6_oDT_R21.fastq.gz
48129839
/tmp/outdata/Scifi_library_2_S2_trimmed-SciFi7_WP_R21.fastq.gz
5420628
/tmp/outdata/Scifi_library_2_S2_trimmed-SciFi7_oDT_R21.fastq.gz
19880142
/tmp/outdata/Scifi_library_2_S2_trimmed-SciFi8_WP_R21.fastq.gz
4423672
/tmp/outdata/Scifi_library_2_S2_trimmed-SciFi8_oDT_R21.fastq.gz
17860185
/tmp/outdata/Scifi_library_2_S2_trimmed-WP1_R21.fastq.gz
452255
/tmp/outdata/Scifi_library_2_S2_trimmed-WP2_R21.fastq.gz
357452
/tmp/outdata/Scifi_library_2_S2_trimmed-WP3_R21.fastq.gz
464263
/tmp/outdata/Scifi_library_2_S2_trimmed-WP4_R21.fastq.gz
1557606
/tmp/outdata/Scifi_library_2_S2_trimmed-oDT1_R21.fastq.gz
494403
/tmp/outdata/Scifi_library_2_S2_trimmed-oDT2_R21.fastq.gz
197908
/tmp/outdata/Scifi_library_2_S2_tri

### Merge all individual files from the same cDNA index

In [14]:
#by same cdna index does it refer to e.g oDT?

#not needed for scifi6 as already done prior!
'''
def catSameID(cDNA_id) :
    print(cDNA_id)
    catSeq = "cat "+export_path+"*oDT_"+cDNA_id+"_R21.fastq.gz > /tmp/outdata/SciFi_oDT_" +cDNA_id+"_R21.fastq.gz"
    catSeqRev = "cat "+export_path+"*oDT_"+cDNA_id+"_R3.fastq.gz > /tmp/outdata/SciFi_oDT_" +cDNA_id+"_R3.fastq.gz"
    print(catSeq)
    !($catSeq)
    !($catSeqRev) 
'''

In [15]:
'''
#Where do these ids come from?
catSameID("A1_5")
catSameID("B1_57")
catSameID("C1_97")
catSameID("D1_137")
catSameID("E1_182")
catSameID("F1_227")
catSameID("G1_266")
catSameID("G2_271")
catSameID("H1_307")
catSameID("H2_314")
'''

A1_5
cat /tmp/outdata/*oDT_A1_5_R21.fastq.gz > /tmp/outdata/SciFi_oDT_A1_5_R21.fastq.gz
cat: '/tmp/outdata/*oDT_A1_5_R21.fastq.gz': No such file or directory
cat: '/tmp/outdata/*oDT_A1_5_R3.fastq.gz': No such file or directory
B1_57
cat /tmp/outdata/*oDT_B1_57_R21.fastq.gz > /tmp/outdata/SciFi_oDT_B1_57_R21.fastq.gz
cat: '/tmp/outdata/*oDT_B1_57_R21.fastq.gz': No such file or directory
cat: '/tmp/outdata/*oDT_B1_57_R3.fastq.gz': No such file or directory
C1_97
cat /tmp/outdata/*oDT_C1_97_R21.fastq.gz > /tmp/outdata/SciFi_oDT_C1_97_R21.fastq.gz
cat: '/tmp/outdata/*oDT_C1_97_R21.fastq.gz': No such file or directory
cat: '/tmp/outdata/*oDT_C1_97_R3.fastq.gz': No such file or directory
D1_137
cat /tmp/outdata/*oDT_D1_137_R21.fastq.gz > /tmp/outdata/SciFi_oDT_D1_137_R21.fastq.gz
cat: '/tmp/outdata/*oDT_D1_137_R21.fastq.gz': No such file or directory
cat: '/tmp/outdata/*oDT_D1_137_R3.fastq.gz': No such file or directory
E1_182
cat /tmp/outdata/*oDT_E1_182_R21.fastq.gz > /tmp/outdata/SciFi_oD

In [16]:
!cat /tmp/outdata/*R21_trimPA.fastq.gz > /tmp/outdata/SciFi_oDT_Complete_R21_trimPA.fastq.gz
!cat /tmp/outdata/*R3_trimPA.fastq.gz > /tmp/outdata/SciFi_oDT_Complete_R3_trimPA.fastq.gz

## Install STAR and align

In [36]:
## The released versions up and including 2.7.9a contain a bug in the STARsolo for quantification. This it needs to be built from pulled from Github and built from source

In [5]:
#! wget https://github.com/alexdobin/STAR/archive/2.7.10a.tar.gz
#! tar -xzf 2.7.10a.tar.gz
#!cd STAR-2.7.10a/source ; make STAR

make -C htslib lib-static
make[1]: Entering directory '/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/STAR-2.7.10a/source/htslib'
cc -g -Wall -O2  -I. -DSAMTOOLS=1 -c -o kfunc.o kfunc.c
cc -g -Wall -O2  -I. -DSAMTOOLS=1 -c -o knetfile.o knetfile.c
cc -g -Wall -O2  -I. -DSAMTOOLS=1 -c -o kstring.o kstring.c
cc -g -Wall -O2  -I. -DSAMTOOLS=1 -c -o bgzf.o bgzf.c
bgzf.c: In function ‘bgzf_write’:
bgzf.c:733:5: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
  733 |     if ( !fp->is_compressed )
      |     ^~
bgzf.c:736:2: note: ...this statement, but the latter is misleadingly indented as if it were guarded by the ‘if’
  736 |  const uint8_t *input = (const uint8_t*)data;
      |  ^~~~~
cc -g -Wall -O2  -I. -DSAMTOOLS=1 -c -o faidx.o faidx.c
faidx.c: In function ‘fai_load’:
faidx.c:265:5: warning: this ‘else’ clause does not guard... [-Wmisleading-indentation]
  265 |     else
      |     ^~~~
faidx.c:268:2: note: ...this statem

cc -g -Wall -O2  -I. -DSAMTOOLS=1 -c -o cram/files.o cram/files.c
cc -g -Wall -O2  -I. -DSAMTOOLS=1 -c -o cram/mFILE.o cram/mFILE.c
cc -g -Wall -O2  -I. -DSAMTOOLS=1 -c -o cram/md5.o cram/md5.c
cc -g -Wall -O2  -I. -DSAMTOOLS=1 -c -o cram/open_trace_file.o cram/open_trace_file.c
cc -g -Wall -O2  -I. -DSAMTOOLS=1 -c -o cram/pooled_alloc.o cram/pooled_alloc.c
cc -g -Wall -O2  -I. -DSAMTOOLS=1 -c -o cram/sam_header.o cram/sam_header.c
In file included from cram/sam_header.c:36:
cram/sam_header.c: In function ‘sam_hdr_add_lines’:
cram/sam_header.c:328:13: warning: suggest parentheses around assignment used as truth value [-Wparentheses]
  328 |      assert(p->next = t);
      |             ^
cram/sam_header.c: In function ‘sam_hdr_vadd’:
cram/sam_header.c:449:9: warning: suggest parentheses around assignment used as truth value [-Wparentheses]
  449 |  assert(p->next = t);
      |         ^
cc -g -Wall -O2  -I. -DSAMTOOLS=1 -c -o cram/string_alloc.o cram/string_alloc.c
cc -g -Wall -O2  -I.

g++: warning: parametersDefault.xxd: linker input file unused because linking not done
g++: warning: htslib: linker input file unused because linking not done
g++ -c   -O3 -std=c++11 -fopenmp -D'COMPILATION_TIME_PLACE="2022-01-17T15:30:52+01:00 :/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/STAR-2.7.10a/source"' -D'GIT_BRANCH_COMMIT_DIFF="On branch main ; commit 360369fb4601a18cc34475b7434e9c042a0e0e01 ; diff files: "' -pipe -Wall -Wextra SoloFeature_collapseUMI_Graph.cpp
g++ -c   -O3 -std=c++11 -fopenmp -D'COMPILATION_TIME_PLACE="2022-01-17T15:30:52+01:00 :/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/STAR-2.7.10a/source"' -D'GIT_BRANCH_COMMIT_DIFF="On branch main ; commit 360369fb4601a18cc34475b7434e9c042a0e0e01 ; diff files: "' -pipe -Wall -Wextra SoloFeature_collapseUMIall_multiMappers.cpp
g++ -c   -O3 -std=c++11 -fopenmp -D'COMPILATION_TIME_PLACE="2022-01-17T15:30:52+01:00 :/media/Dropbox/MNM projects/sn

g++ -c   -O3 -std=c++11 -fopenmp -D'COMPILATION_TIME_PLACE="2022-01-17T15:30:52+01:00 :/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/STAR-2.7.10a/source"' -D'GIT_BRANCH_COMMIT_DIFF="On branch main ; commit 360369fb4601a18cc34475b7434e9c042a0e0e01 ; diff files: "' -pipe -Wall -Wextra ParametersSolo.cpp
g++ -c   -O3 -std=c++11 -fopenmp -D'COMPILATION_TIME_PLACE="2022-01-17T15:30:52+01:00 :/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/STAR-2.7.10a/source"' -D'GIT_BRANCH_COMMIT_DIFF="On branch main ; commit 360369fb4601a18cc34475b7434e9c042a0e0e01 ; diff files: "' -pipe -Wall -Wextra SoloRead.cpp
g++ -c   -O3 -std=c++11 -fopenmp -D'COMPILATION_TIME_PLACE="2022-01-17T15:30:52+01:00 :/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/STAR-2.7.10a/source"' -D'GIT_BRANCH_COMMIT_DIFF="On branch main ; commit 360369fb4601a18cc34475b7434e9c042a0e0e01 ; diff files: "' -pipe -Wall -Wextra

g++ -c   -O3 -std=c++11 -fopenmp -D'COMPILATION_TIME_PLACE="2022-01-17T15:30:52+01:00 :/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/STAR-2.7.10a/source"' -D'GIT_BRANCH_COMMIT_DIFF="On branch main ; commit 360369fb4601a18cc34475b7434e9c042a0e0e01 ; diff files: "' -pipe -Wall -Wextra ReadAlign_mapOneReadSpliceGraph.cpp
g++ -c   -O3 -std=c++11 -fopenmp -D'COMPILATION_TIME_PLACE="2022-01-17T15:30:52+01:00 :/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/STAR-2.7.10a/source"' -D'GIT_BRANCH_COMMIT_DIFF="On branch main ; commit 360369fb4601a18cc34475b7434e9c042a0e0e01 ; diff files: "' -pipe -Wall -Wextra SpliceGraph.cpp
g++ -c   -O3 -std=c++11 -fopenmp -D'COMPILATION_TIME_PLACE="2022-01-17T15:30:52+01:00 :/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/STAR-2.7.10a/source"' -D'GIT_BRANCH_COMMIT_DIFF="On branch main ; commit 360369fb4601a18cc34475b7434e9c042a0e0e01 ; diff files: "'

g++ -c   -O3 -std=c++11 -fopenmp -D'COMPILATION_TIME_PLACE="2022-01-17T15:30:52+01:00 :/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/STAR-2.7.10a/source"' -D'GIT_BRANCH_COMMIT_DIFF="On branch main ; commit 360369fb4601a18cc34475b7434e9c042a0e0e01 ; diff files: "' -pipe -Wall -Wextra Transcript_alignScore.cpp
g++ -c   -O3 -std=c++11 -fopenmp -D'COMPILATION_TIME_PLACE="2022-01-17T15:30:52+01:00 :/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/STAR-2.7.10a/source"' -D'GIT_BRANCH_COMMIT_DIFF="On branch main ; commit 360369fb4601a18cc34475b7434e9c042a0e0e01 ; diff files: "' -pipe -Wall -Wextra Transcript_generateCigarP.cpp
g++ -c   -O3 -std=c++11 -fopenmp -D'COMPILATION_TIME_PLACE="2022-01-17T15:30:52+01:00 :/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/STAR-2.7.10a/source"' -D'GIT_BRANCH_COMMIT_DIFF="On branch main ; commit 360369fb4601a18cc34475b7434e9c042a0e0e01 ; diff files

g++ -c   -O3 -std=c++11 -fopenmp -D'COMPILATION_TIME_PLACE="2022-01-17T15:30:52+01:00 :/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/STAR-2.7.10a/source"' -D'GIT_BRANCH_COMMIT_DIFF="On branch main ; commit 360369fb4601a18cc34475b7434e9c042a0e0e01 ; diff files: "' -pipe -Wall -Wextra ReadAlign_createExtendWindowsWithAlign.cpp
g++ -c   -O3 -std=c++11 -fopenmp -D'COMPILATION_TIME_PLACE="2022-01-17T15:30:52+01:00 :/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/STAR-2.7.10a/source"' -D'GIT_BRANCH_COMMIT_DIFF="On branch main ; commit 360369fb4601a18cc34475b7434e9c042a0e0e01 ; diff files: "' -pipe -Wall -Wextra ReadAlign_assignAlignToWindow.cpp
g++ -c   -O3 -std=c++11 -fopenmp -D'COMPILATION_TIME_PLACE="2022-01-17T15:30:52+01:00 :/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/STAR-2.7.10a/source"' -D'GIT_BRANCH_COMMIT_DIFF="On branch main ; commit 360369fb4601a18cc34475b7434e9c04

g++ -c   -O3 -std=c++11 -fopenmp -D'COMPILATION_TIME_PLACE="2022-01-17T15:30:52+01:00 :/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/STAR-2.7.10a/source"' -D'GIT_BRANCH_COMMIT_DIFF="On branch main ; commit 360369fb4601a18cc34475b7434e9c042a0e0e01 ; diff files: "' -pipe -Wall -Wextra insertSeqSA.cpp
g++ -c   -O3 -std=c++11 -fopenmp -D'COMPILATION_TIME_PLACE="2022-01-17T15:30:52+01:00 :/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/STAR-2.7.10a/source"' -D'GIT_BRANCH_COMMIT_DIFF="On branch main ; commit 360369fb4601a18cc34475b7434e9c042a0e0e01 ; diff files: "' -pipe -Wall -Wextra funCompareUintAndSuffixes.cpp
g++ -c   -O3 -std=c++11 -fopenmp -D'COMPILATION_TIME_PLACE="2022-01-17T15:30:52+01:00 :/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/STAR-2.7.10a/source"' -D'GIT_BRANCH_COMMIT_DIFF="On branch main ; commit 360369fb4601a18cc34475b7434e9c042a0e0e01 ; diff files: "' -pipe

g++ -c   -O3 -std=c++11 -fopenmp -D'COMPILATION_TIME_PLACE="2022-01-17T15:30:52+01:00 :/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/STAR-2.7.10a/source"' -D'GIT_BRANCH_COMMIT_DIFF="On branch main ; commit 360369fb4601a18cc34475b7434e9c042a0e0e01 ; diff files: "' -pipe -Wall -Wextra BAMfunctions.cpp
g++ -c   -O3 -std=c++11 -fopenmp -D'COMPILATION_TIME_PLACE="2022-01-17T15:30:52+01:00 :/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/STAR-2.7.10a/source"' -D'GIT_BRANCH_COMMIT_DIFF="On branch main ; commit 360369fb4601a18cc34475b7434e9c042a0e0e01 ; diff files: "' -pipe -Wall -Wextra ReadAlign_alignBAM.cpp
g++ -c   -O3 -std=c++11 -fopenmp -D'COMPILATION_TIME_PLACE="2022-01-17T15:30:52+01:00 :/media/Dropbox/MNM projects/snRNA-seq Synuclein project/Data analysis/scifi-analysis/STAR-2.7.10a/source"' -D'GIT_BRANCH_COMMIT_DIFF="On branch main ; commit 360369fb4601a18cc34475b7434e9c042a0e0e01 ; diff files: "' -pipe -Wall

In [ ]:
#get the trimmed R3 and R1 files, save into a dict with R3 being the key and R1 being the corresponding value. 
#Iterate over all of the key-value pairs and pass them as input files in STARsolo, thus processing them 1 by one.


In [ ]:
#!./runStarSolo.sh
!./runSTARsolo_orig.sh

mkdir: cannot create directory ‘/media/data/AtteR/scifi-analysis/outputs_starsolo’: File exists
	STAR --genomeDir /media/data/AtteR/scifi-analysis/ref_genome/rat_index --runThreadN 24 --soloType CB_UMI_Simple --soloCBwhitelist /media/data/AtteR/scifi-analysis/737K-cratac-v1.txt --soloCBstart 1 --soloCBlen 16 --soloUMIstart 17 --soloUMIlen 8 --soloBarcodeReadLength 0 --soloFeatures GeneFull --readFilesCommand zcat --soloMultiMappers Uniform EM PropUnique Unique --soloUMIdedup 1MM_All --soloCBmatchWLtype 1MM_multi_Nbase_pseudocounts --genomeLoad LoadAndKeep --alignIntronMin 20 --alignIntronMax 1000000 --readFilesIn /media/data/AtteR/scifi-analysis/pilot1_output_trim/Scifi_library_2_S2_trimmed-SciFi5_oDT_R3.fastq.gz /media/data/AtteR/scifi-analysis/pilot1_output_trim/Scifi_library_2_S2_trimmed-SciFi5_oDT_R21.fastq.gz
	STAR version: 2.7.9a_2021-06-25   compiled: 2021-12-21T15:48:20+01:00 :/home/lcadmin/STAR/source
Feb 10 15:05:14 ..... started STAR run
Feb 10 15:05:15 ..... loading genome


## Generate a summary tables

In [39]:
star_path = "/media/data/AtteR/scifi-analysis/pilot1_output/"
star_path = "/media/data/AtteR/scifi-analysis/outputs_starsolo/"

In [40]:
#samples_list = [dir for dir in sorted(glob.glob(star_path +"*")) if os.path.isdir(dir)] 

#samples_list = os.listdir(star_path)
samples_list = []
for subdir in os.listdir(star_path):
    full_path = star_path + subdir
    samples_list.append(full_path)
samples_list

['/media/data/AtteR/scifi-analysis/outputs_starsolo/Scifi_library_2_S2_SciFi5_oDT',
 '/media/data/AtteR/scifi-analysis/outputs_starsolo/Scifi_library_2_S2_SciFi7_oDT',
 '/media/data/AtteR/scifi-analysis/outputs_starsolo/Scifi_library_2_S2_SciFi8_oDT',
 '/media/data/AtteR/scifi-analysis/outputs_starsolo/Scifi_library_2_S2_SciFi6_oDT']

In [41]:
summaryTable = pd.DataFrame()

In [18]:
for sample_id in samples_list :
    print(sample_id)
    df = pd.read_csv(sample_id + "/Log.final.out", sep='\t', header=None, index_col=0)
    print(df)
    df = df.rename(columns={1: ntpath.basename(sample_id)})
    summaryTable = summaryTable.merge(df,left_index=True, right_index=True, how='outer')

/tmp/outdata/SciFi_new_cDNA_Pool1_R21_trimPA.fastq.gz


NotADirectoryError: [Errno 20] Not a directory: '/tmp/outdata/SciFi_new_cDNA_Pool1_R21_trimPA.fastq.gz/Log.final.out'

In [17]:
summaryTable

NameError: name 'summaryTable' is not defined

In [57]:
from datetime import date
run_date = date.today()
print(run_date)
print(output_tables)
summaryTable.to_csv(path_or_buf=output_tables + '/ScifiPilot1_STARsoloSummary_' + str(run_date) + '.csv', sep=',', header=True, index=True, index_label='Variable')

2022-01-28
/media/data/AtteR/scifi-analysis/summary_tables


### Generate second summary

In [58]:
summaryTable = pd.DataFrame()

In [59]:
for sample_id in samples_list :
    print(sample_id)
    df = pd.read_csv(sample_id + "/GeneFull/Summary.csv", sep=',', header=None, index_col=0)
    df = df.rename(columns={1: ntpath.basename(sample_id)})
    summaryTable = summaryTable.merge(df,left_index=True, right_index=True, how='outer')

/media/data/AtteR/scifi-analysis/outputs_starsolo/Scifi_library_2_S2_SciFi5_oDT
/media/data/AtteR/scifi-analysis/outputs_starsolo/Scifi_library_2_S2_SciFi7_oDT
/media/data/AtteR/scifi-analysis/outputs_starsolo/Scifi_library_2_S2_SciFi8_oDT
/media/data/AtteR/scifi-analysis/outputs_starsolo/Scifi_library_2_S2_SciFi6_oDT


In [60]:
summaryTable

,Scifi_library_2_S2_SciFi5_oDT,Scifi_library_2_S2_SciFi7_oDT,Scifi_library_2_S2_SciFi8_oDT,Scifi_library_2_S2_SciFi6_oDT
0,,,,
Number of Reads,1.951169e+08,1.988014e+07,1.786018e+07,4.812984e+07
Reads With Valid Barcodes,9.048160e-01,9.076040e-01,8.913700e-01,9.016370e-01
Sequencing Saturation,8.716100e-01,9.017350e-01,9.117860e-01,9.109690e-01
Q30 Bases in CB+UMI,9.060180e-01,9.188270e-01,9.147440e-01,9.167430e-01
Q30 Bases in RNA read,9.156260e-01,9.152880e-01,9.118650e-01,9.135630e-01
Reads Mapped to Genome: Unique+Multiple,8.232480e-01,6.736000e-01,7.762160e-01,8.040400e-01
Reads Mapped to Genome: Unique,6.065520e-01,5.318930e-01,6.259740e-01,5.669460e-01
Reads Mapped to GeneFull: Unique+Multipe GeneFull,6.499750e-01,5.111110e-01,5.801640e-01,6.235110e-01
Reads Mapped to GeneFull: Unique GeneFull,4.775710e-01,4.034620e-01,4.670540e-01,4.290440e-01


In [61]:
from datetime import date
run_date = date.today()
print(run_date)
print(output_tables)
summaryTable.to_csv(path_or_buf=output_tables + '/ScifiPilot1_STARsoloGeneFul_' + str(run_date) + '.csv', sep=',', header=True, index=True, index_label='Variable')

2022-01-28
/media/data/AtteR/scifi-analysis/summary_tables


## Analyze AAV sequences

In [40]:
samples_list = sorted(glob.glob(base_path+'*AAV*R21*.fastq.gz'))
samples_list

['/tmp/indata/SciFi_Kapa_AAV_aSyn_S6_R21_001.fastq.gz',
 '/tmp/indata/SciFi_Kapa_AAV_tagBFP_S5_R21_001.fastq.gz']

In [45]:
def trimBC(sample_id) :
    print(sample_id)
    #sample_id = trimSamples_list[10]
    inPathR2 = base_path + ntpath.basename(sample_id).replace('_R21_001.fastq.gz','_R3_001.fastq.gz')
    outPathR1 = export_path + ntpath.basename(sample_id).replace('_R21_001.fastq.gz','_AAV-{name}_R21.fastq.gz')
    outPathR2 = export_path + ntpath.basename(sample_id).replace('_R21_001.fastq.gz','_AAV-{name}_R3.fastq.gz')
    
    trimLog = export_path + ntpath.basename(sample_id).replace('_R21_001.fastq.gz','_AAV-Log.tsv')
    param = "--cores=0 -g aSyn=ACTACGAACCTGAAGCCTAAAGG...CCTAATCAACCTCTGGATTACAAAATTTGTG -g tagBFP=AACTGGGGCACAAGCTTAATTAA...AGGCCTAATCAACCTCTGGATTACAAAATTT --trimmed-only --report=minimal"
    call_sequence = "cutadapt "+param+" -o "+outPathR2+" -p " + outPathR1 + " "+ inPathR2 + " " + sample_id +" > " + trimLog
    !$call_sequence
    print(call_sequence)
    #call_sequence    

In [46]:
[trimBC(sample_id) for sample_id in samples_list]

/tmp/indata/SciFi_Kapa_AAV_aSyn_S6_R21_001.fastq.gz
Done           00:00:14     2,822,255 reads @   5.1 µs/read;  11.85 M reads/minute
cutadapt --cores=0 -g aSyn=ACTACGAACCTGAAGCCTAAAGG...CCTAATCAACCTCTGGATTACAAAATTTGTG -g tagBFP=AACTGGGGCACAAGCTTAATTAA...AGGCCTAATCAACCTCTGGATTACAAAATTT --trimmed-only --report=minimal -o /tmp/outdata/SciFi_Kapa_AAV_aSyn_S6_AAV-{name}_R3.fastq.gz -p /tmp/outdata/SciFi_Kapa_AAV_aSyn_S6_AAV-{name}_R21.fastq.gz /tmp/indata/SciFi_Kapa_AAV_aSyn_S6_R3_001.fastq.gz /tmp/indata/SciFi_Kapa_AAV_aSyn_S6_R21_001.fastq.gz > /tmp/outdata/SciFi_Kapa_AAV_aSyn_S6_AAV-Log.tsv
/tmp/indata/SciFi_Kapa_AAV_tagBFP_S5_R21_001.fastq.gz
Done           00:00:08     1,651,948 reads @   5.0 µs/read;  12.06 M reads/minute
cutadapt --cores=0 -g aSyn=ACTACGAACCTGAAGCCTAAAGG...CCTAATCAACCTCTGGATTACAAAATTTGTG -g tagBFP=AACTGGGGCACAAGCTTAATTAA...AGGCCTAATCAACCTCTGGATTACAAAATTT --trimmed-only --report=minimal -o /tmp/outdata/SciFi_Kapa_AAV_tagBFP_S5_AAV-{name}_R3.fastq.gz -p /tmp/outdata/

[None, None]

In [47]:
qc_list = sorted(glob.glob(export_path+'*AAV-Log.tsv'))
allSampleTable = pd.DataFrame()

for qc_file in qc_list :
    df=pd.read_csv(qc_file, sep='\t') # , header=0, index_col=0,skiprows=None
    df = df.set_index(pd.Series(ntpath.basename(qc_file).replace('_trimLog.tsv','')))
    allSampleTable = pd.concat([allSampleTable, df], axis=0)

allSampleTable

,status,in_reads,in_bp,too_short,too_long,too_many_n,out_reads,w/adapters,qualtrim_bp,out_bp,w/adapters2,qualtrim2_bp,out2_bp
SciFi_Kapa_AAV_aSyn_S6_AAV-Log.tsv,OK,2822255,445916290,0,0,0,2648737,2648737,0,63263488,0,0,113895691
SciFi_Kapa_AAV_tagBFP_S5_AAV-Log.tsv,OK,1651948,261007784,0,0,0,1538839,1538839,0,37967841,0,0,66170077


In [48]:
samples_list = sorted(glob.glob(export_path+'*AAV*.fastq.gz'))
for sample_file in samples_list :
    print(sample_file)
    result = subprocess.run(['pigz -dc '+ sample_file+ ' | wc -l'], stdout=subprocess.PIPE, shell=True)
    sampleReads = int(int(result.stdout)/4)
    if sampleReads < 4000 :
        os.remove(sample_file)
    print(sampleReads)

/tmp/outdata/SciFi_Kapa_AAV_aSyn_S6_AAV-aSyn_R21.fastq.gz
2648732
/tmp/outdata/SciFi_Kapa_AAV_aSyn_S6_AAV-aSyn_R3.fastq.gz
2648732
/tmp/outdata/SciFi_Kapa_AAV_aSyn_S6_AAV-tagBFP_R21.fastq.gz
5
/tmp/outdata/SciFi_Kapa_AAV_aSyn_S6_AAV-tagBFP_R3.fastq.gz
5
/tmp/outdata/SciFi_Kapa_AAV_tagBFP_S5_AAV-aSyn_R21.fastq.gz
364
/tmp/outdata/SciFi_Kapa_AAV_tagBFP_S5_AAV-aSyn_R3.fastq.gz
364
/tmp/outdata/SciFi_Kapa_AAV_tagBFP_S5_AAV-tagBFP_R21.fastq.gz
1538475
/tmp/outdata/SciFi_Kapa_AAV_tagBFP_S5_AAV-tagBFP_R3.fastq.gz
1538475


In [72]:
AAV_list = sorted(glob.glob(export_path+'*AAV-aSyn_R3.fastq.gz'))
AAV_list

['/tmp/outdata/SciFi_Kapa_AAV_aSyn_S6_AAV-aSyn_R3.fastq.gz']

In [61]:
def runSC(sample_id) :
    print(sample_id)
    
    outPath = export_path + ntpath.basename(sample_id).replace('_R3.fastq.gz','_starcode.tsv')
    starcode_call= "gunzip -c '"+ sample_id +"' | /media/data/TomasB/starcode/starcode -t 48 -o '"+outPath + "'"
    !$starcode_call 

In [73]:
[runSC(sample_id) for sample_id in AAV_list]

/tmp/outdata/SciFi_Kapa_AAV_aSyn_S6_AAV-aSyn_R3.fastq.gz
running starcode with 48 threads
reading input files
FASTQ format detected
sorting
setting dist to 2
progress: 100.00%
message passing clustering


[None]

In [74]:
AAV_SC_list = sorted(glob.glob(export_path+'*AAV-aSyn_starcode.tsv'))
AAV_SC_list

['/tmp/outdata/SciFi_Kapa_AAV_aSyn_S6_AAV-aSyn_starcode.tsv']

In [75]:
starcodeTable = pd.DataFrame()
for sample_id in AAV_SC_list :
    df = pd.read_csv(sample_id, sep='\t', header=None, index_col=0, dtype={0: str, 1: 'Int64'})
    df = df.rename(columns={1: ntpath.basename(sample_id).replace('_starcode.tsv','')})
    starcodeTable = starcodeTable.merge(df,left_index=True, right_index=True, how='outer')

In [76]:
starcodeTable

,SciFi_Kapa_AAV_aSyn_S6_AAV-aSyn
0,
ACATCCATGCGTACATCATTCCTT,61980
GAGCGCTGGAGGGAGGACAGACGT,59413
ATATGCGGCAGTGTTCATTGATTC,55206
CCTGAATCGCGTGAACCCTTGAAT,49405
ATGGACTGGTACACAGGCTGATGT,46600
...,...
TTTTCCGCATGCAATCCTTGCTAT,1
TTTTGCGGCAGGGTTCATTTATTC,1
TTTTGCTCGATGATATATACTATG,1


In [77]:
starcodeTable = starcodeTable[starcodeTable['SciFi_Kapa_AAV_aSyn_S6_AAV-aSyn']>2]

In [78]:
allBCsCount = starcodeTable.count()

In [79]:
allBCsCount

SciFi_Kapa_AAV_aSyn_S6_AAV-aSyn    296
dtype: int64

In [166]:
samples_list = sorted(glob.glob(base_path+'*R1*.fastq.gz'))
#samples_list= [item for item in samples_list if 'Undetermined' not in item]
#samples_list= [item for item in samples_list if 'BC' in item]
samples_list

['/tmp/indata/SciFi_Kapa_AAV_aSyn_S6_R1_001.fastq.gz',
 '/tmp/indata/SciFi_Kapa_AAV_tagBFP_S5_R1_001.fastq.gz',
 '/tmp/indata/SciFi_Kapa_input_S2_R1_001.fastq.gz',
 '/tmp/indata/SciFi_Phusion_TAG1_N712_S3_R1_001.fastq.gz',
 '/tmp/indata/SciFi_Phusion_TAG1_N715_S4_R1_001.fastq.gz',
 '/tmp/indata/SciFi_Phusion_input_S1_R1_001.fastq.gz']

In [140]:
def runSC(sample_id) :
    print(sample_id)
    #outPath = re.sub('R*[1-3]*_001.fastq.gz', 'starcode.tsv', export_path + ntpath.basename(sample_id), 2)
    outPath = export_path + ntpath.basename(sample_id).replace('_trimPA.fastq.gz','_starcode.tsv')
    starcode_call= "gunzip -c '"+ sample_id +"' | /media/data/TomasB/starcode/starcode -t 48 -o '"+outPath + "'"
    !$starcode_call 

In [149]:
samples_list = sorted(glob.glob(export_path+'*R3_trimPA.fastq.gz'))
[runSC(sample_id) for sample_id in samples_list]

/tmp/outdata/SciFi_Kapa_input_S2_R1_trimPA.fastq.gz
running starcode with 48 threads
reading input files
FASTQ format detected
sorting
setting dist to 2
progress: 100.00%
message passing clustering
/tmp/outdata/SciFi_Kapa_input_S2_R3_trimPA.fastq.gz
running starcode with 48 threads
reading input files
FASTQ format detected
sorting
setting dist to 2
progress: 100.00%
message passing clustering
/tmp/outdata/SciFi_Phusion_TAG2_N714_S3_R1_trimPA.fastq.gz
running starcode with 48 threads
reading input files
FASTQ format detected
sorting
setting dist to 2
progress: 100.00%
message passing clustering
^C
/tmp/outdata/SciFi_Phusion_TAG2_N714_S3_R3_trimPA.fastq.gz
running starcode with 48 threads
reading input files
FASTQ format detected
sorting
setting dist to 2
progress: 100.00%
message passing clustering
/tmp/outdata/SciFi_Phusion_TAG2_N715_S4_R1_trimPA.fastq.gz
running starcode with 48 threads
reading input files
FASTQ format detected
sorting
setting dist to 2
progress: 100.00%
message passi

[None, None, None, None, None, None, None, None]

In [150]:
R3_list = sorted(glob.glob(export_path+'*R3_starcode.tsv'))
R3_list

['/tmp/outdata/SciFi_Kapa_AAV_aSyn_S6_R3_starcode.tsv',
 '/tmp/outdata/SciFi_Kapa_AAV_tagBFP_S5_R3_starcode.tsv',
 '/tmp/outdata/SciFi_Kapa_input_S2_R3_starcode.tsv',
 '/tmp/outdata/SciFi_Phusion_TAG2_N714_S3_R3_starcode.tsv',
 '/tmp/outdata/SciFi_Phusion_TAG2_N715_S4_R3_starcode.tsv',
 '/tmp/outdata/SciFi_Phusion_input_S1_R3_starcode.tsv']

In [151]:
starcodeTable = pd.DataFrame()
for sample_id in R3_list[2:] :
    df = pd.read_csv(sample_id, sep='\t', header=None, index_col=0, dtype={0: str, 1: 'Int64'}, nrows=15)
    df = df.rename(columns={1: ntpath.basename(sample_id).replace('_starcode.tsv','')})
    starcodeTable = starcodeTable.merge(df,left_index=True, right_index=True, how='outer')

In [152]:
starcodeTable

,SciFi_Kapa_input_S2_R3,SciFi_Phusion_TAG2_N714_S3_R3,SciFi_Phusion_TAG2_N715_S4_R3,SciFi_Phusion_input_S1_R3
0,,,,
NaN,4050511,1495732,1598900,53551
GCTGC,3693705,820514,920244,28857
GCTGCACC,420898,141028,153227,<NA>
GCTGCACCAAAAAAAAAAAAAAAAA,524815,129951,<NA>,4391
GCTGCACCGAAAAAAAAAAAAAAAA,<NA>,167672,179724,5298
GCTGCACCGTC,14706457,4238130,4618962,133826
GCTGCACCGTCAAAAAAAAAAAAAA,1073009,413310,461689,11084
GCTGCACCGTCAG,3103904,929411,1009270,26607
GCTGCACCGTCAGAAAAAAAAAAAA,1381805,354859,371144,13404


In [153]:
starcodeTable.to_csv(path_or_buf=export_path+ 'R3_starcodeTable.tsv', sep='\t', header=True, index=True, index_label='Sequence')

In [102]:
def runCollapser(sample_id) :
    print(sample_id)
    outPath = export_path + ntpath.basename(sample_id).replace('_001.fastq.gz','_Collapsed.fastq.gz')
    outClusterPath = export_path + ntpath.basename(sample_id).replace('_001.fastq.gz','_Clusters.txt')
    collapser_call= "dedupe.sh in='"+ sample_id +"' out='"+outPath +"' csf='" + outClusterPath + "'"
    !$collapser_call 

In [103]:
samples_list = sorted(glob.glob(base_path+'*R3*.fastq.gz'))
[runCollapser(sample_id) for sample_id in samples_list]

/tmp/indata/SciFi_Kapa_AAV_aSyn_S6_R3_001.fastq.gz
java -ea -Xmx204824m -Xms204824m -cp /usr/local/bin/bbmap/current/ jgi.Dedupe in=/tmp/indata/SciFi_Kapa_AAV_aSyn_S6_R3_001.fastq.gz out=/tmp/outdata/SciFi_Kapa_AAV_aSyn_S6_R3_Collapsed.fastq.gz csf=/tmp/outdata/SciFi_Kapa_AAV_aSyn_S6_R3_Clusters.txt
Executing jgi.Dedupe [in=/tmp/indata/SciFi_Kapa_AAV_aSyn_S6_R3_001.fastq.gz, out=/tmp/outdata/SciFi_Kapa_AAV_aSyn_S6_R3_Collapsed.fastq.gz, csf=/tmp/outdata/SciFi_Kapa_AAV_aSyn_S6_R3_Clusters.txt]
Version 38.93

Initial:
Memory: max=214781m, total=214781m, free=214731m, used=50m

Found 1154137 duplicates.
Finished exact matches.    Time: 4.362 seconds.
Memory: max=214781m, total=214781m, free=211409m, used=3372m

Found 0 contained sequences.
Finished containment.      Time: 5.426 seconds.
Memory: max=214781m, total=214781m, free=210554m, used=4227m

Removed 0 invalid entries.
Finished invalid removal.  Time: 0.092 seconds.
Memory: max=214781m, total=214781m, free=210554m, used=4227m

Input:

[None, None, None, None, None, None]

In [110]:
def runBBmerge(sample_id) :
    print(sample_id)
    unmatched = export_path + ntpath.basename(sample_id).replace('_001.fastq.gz','_unMatched.fastq.gz')
    samplePair = base_path + ntpath.basename(sample_id).replace('R1_001.fastq.gz','R3_001.fastq.gz')
    unmatchedPair= export_path + ntpath.basename(samplePair).replace('_001.fastq.gz','_unMatched.fastq.gz')
    outPath = export_path + ntpath.basename(sample_id).replace('_001.fastq.gz','_MergedPair.fastq.gz')
    
    outhistogram = export_path + ntpath.basename(sample_id).replace('_001.fastq.gz','_histogram.txt')
    merge_call= "bbmerge.sh in1='"+ sample_id +"' in2='"+samplePair +"' out='"+outPath +"' outu1='"+unmatched +"' outu2='"+unmatchedPair +"' ihist='" + outhistogram + "' adapter1=AATGATACGGCGACCACCGAGATCTACAC adapter2=CAAGCAGAAGACGGCATACGAGAT"
    !$merge_call 

In [112]:
samples_list = sorted(glob.glob(base_path+'*R1*.fastq.gz'))
[runBBmerge(sample_id) for sample_id in samples_list[2:3]]

/tmp/indata/SciFi_Kapa_input_S2_R1_001.fastq.gz
java -ea -Xmx1000m -Xms1000m -Djava.library.path=/usr/local/bin/bbmap/jni/ -cp /usr/local/bin/bbmap/current/ jgi.BBMerge in1=/tmp/indata/SciFi_Kapa_input_S2_R1_001.fastq.gz in2=/tmp/indata/SciFi_Kapa_input_S2_R3_001.fastq.gz out=/tmp/outdata/SciFi_Kapa_input_S2_R1_MergedPair.fastq.gz outu1=/tmp/outdata/SciFi_Kapa_input_S2_R1_unMatched.fastq.gz outu2=/tmp/outdata/SciFi_Kapa_input_S2_R3_unMatched.fastq.gz ihist=/tmp/outdata/SciFi_Kapa_input_S2_R1_histogram.txt adapter1=AATGATACGGCGACCACCGAGATCTACAC adapter2=CAAGCAGAAGACGGCATACGAGAT
Executing jgi.BBMerge [in1=/tmp/indata/SciFi_Kapa_input_S2_R1_001.fastq.gz, in2=/tmp/indata/SciFi_Kapa_input_S2_R3_001.fastq.gz, out=/tmp/outdata/SciFi_Kapa_input_S2_R1_MergedPair.fastq.gz, outu1=/tmp/outdata/SciFi_Kapa_input_S2_R1_unMatched.fastq.gz, outu2=/tmp/outdata/SciFi_Kapa_input_S2_R3_unMatched.fastq.gz, ihist=/tmp/outdata/SciFi_Kapa_input_S2_R1_histogram.txt, adapter1=AATGATACGGCGACCACCGAGATCTACAC, adapt

[None]

In [ ]:
bbmerge-auto.sh in=reads.fq out=merged.fq adapter1=AATGATACGGCGACCACCGAGATCTACAC adapter2=CAAGCAGAAGACGGCATACGAGAT rem k=62 extend2=50 ecct



bbmerge.sh  in1= out= outu1=unmerged.fq ihist=ihist.txt


reformat.sh

fuse.sh in1=read1.fq in2=read2.fq out=fused.fq fusepairs pad=14

In [96]:
R1_list = sorted(glob.glob(export_path+'*R1_starcode.tsv'))
R1_list

starcodeTable = pd.DataFrame()
for sample_id in R1_list :
    df = pd.read_csv(sample_id, sep='\t', header=None, index_col=0, dtype={0: str, 1: 'Int64'})
    df = df[0:10] #Analyze just the 10 most common
    df = df.rename(columns={1: ntpath.basename(sample_id).replace('_starcode.tsv','')})
    starcodeTable = starcodeTable.merge(df,left_index=True, right_index=True, how='outer')

NameError: name 'AAV_SC_list' is not defined

In [ ]:
starcodeTable

In [ ]:
STAR 
--genomeDir /home/genomes/rat/STARidx77 
--runThreadN 24 
--soloType CB_UMI_Simple 
--soloCBwhitelist /home/data/soloWhitelist.txt 
--soloCBstart 9 
--soloCBlen 13 
--soloUMIstart 1 
--soloUMIlen 8 
--soloBarcodeReadLength 0 
--soloFeatures GeneFull 
--readFilesCommand zcat 
--soloMultiMappers PropUnique --outSAMtype BAM SortedByCoordinate --outSAMattributes UR CB UB GX GN --limitBAMsortRAM 220000000000 --soloCellFilter TopCells 1 0.99 10 --soloUMIdedup 1MM_All --soloCBmatchWLtype 1MM_multi_Nbase_pseudocounts --genomeLoad LoadAndKeep --soloOutFileNames Scifi_library_SciFi5/ features.tsv barcodes.tsv matrix.mtx --alignIntronMin 20 --alignIntronMax 1000000 --readFilesIn /home/data/output_cutadapt_novaseq/Scifi_library_trimmed-SciFi5_oDT_R3.fastq.gz /home/data/output_cutadapt_novaseq/Scifi_library_trimmed-SciFi5_oDT_R1.fastq.gz